## SB1 
* Need to find out
    * What's the difference between RCA/RCA_Projects_092023 (FeatureServer) and RCA/RCA_Projects_102023 (FeatureServer) here https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA?
    * What's the right link to use to grab the projects: https://odpsvcs.dot.ca.gov/arcgis/rest/services/SB1/SB1_ProjectData/FeatureServer or https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA?
        * I think link 1 is related to the SB1 map online
        * Unsure what the second map is about?

In [1]:
import _harmonization_utils
import _sb1_utils
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis import utils
from calitp_data_analysis.sql import to_snakecase

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Looking at duplicated projectid & objectid

In [3]:
project_table = _sb1_utils.load_sb1()

/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gdf[i]
/home/jovyan/data-analyses/project_list/_sb1_utils.py:23: FutureWarning: The default value of regex will change from True to False in a future version.
  gdf[i]


0 rows are mising geometry
7917 rows contain invalid geography


In [4]:
dont_preview = ["geometry", "description"]

In [5]:
project_table.loc[project_table.projectid == "0715000051"].drop(columns=dont_preview)

,projectid,projname,projcatcode,projcategory,projprogcode,projprogram,multiprogfunded,projstatus,cost,assemblydistrict,senatedistrict,assemblycode,senatecode,countyname,cityname,countycode,citycode,appagencyname,impagencyname,totalcosts,routes,constyear,costfull,projagency
924,0715000051,Los Angeles 110 2 5 405 Traffic Management System Installation Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,38500000,43 51 53 59 62 64 66 70,24 25 26 30 35,4351535962646670,2425263035,Los Angeles,Carson Glendale Hawthorne Los Angeles,La,Crsngndlhawla,NaN,NaN,NaN,25110405,2019,38547000.00,Caltrans
357,0715000051,Los Angeles 110 2 5 405 Traffic Management System Installation Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,38500000,43 51 53 59 62 64 66 70,24 25 26 30 35,4351535962646670,2425263035,Los Angeles,Carson Glendale Hawthorne Los Angeles,La,Crsngndlhawla,NaN,NaN,NaN,25110405,2019,38547000.00,Caltrans
9001,0715000051,Los Angeles 110 2 5 405 Traffic Management System Installation Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,38500000,43 51 53 59 62 64 66 70,24 25 26 30 35,4351535962646670,2425263035,Los Angeles,Carson Glendale Hawthorne Los Angeles,La,Crsngndlhawla,NaN,NaN,NaN,25110405,2019,38547000.00,Caltrans


In [6]:
project_table.loc[project_table.projectid == "0717000060"].drop(columns=dont_preview)

,projectid,projname,projcatcode,projcategory,projprogcode,projprogram,multiprogfunded,projstatus,cost,assemblydistrict,senatedistrict,assemblycode,senatecode,countyname,cityname,countycode,citycode,appagencyname,impagencyname,totalcosts,routes,constyear,costfull,projagency
9025,0717000060,Los Angeles 2 5 91 405 Upgrade Traffic Management System Mobility Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,24700000,43 51 57 58 62 63 64 65 66,24 25 26 29 32 33 35,435157586263646566,24252629323335,Los Angeles,NaN,La,NaN,NaN,NaN,NaN,2591405,NaN,24697000.00,Caltrans
941,0717000060,Los Angeles 2 5 91 405 Upgrade Traffic Management System Mobility Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,24700000,43 51 57 58 62 63 64 65 66,24 25 26 29 32 33 35,435157586263646566,24252629323335,Los Angeles,,La,,NaN,NaN,NaN,2591405,NaN,24697000.00,Caltrans
377,0717000060,Los Angeles 2 5 91 405 Upgrade Traffic Management System Mobility Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,24700000,43 51 57 58 62 63 64 65 66,24 25 26 29 32 33 35,435157586263646566,24252629323335,Los Angeles,,La,,NaN,NaN,NaN,2591405,NaN,24697000.00,Caltrans


In [7]:
# sb1_df1.loc[sb1_df1.projectid == "0200020286"].drop(columns = dont_preview)

In [8]:
len(project_table)

9186

#### Once I want to concat Sb1 with the other data sources, I need one row = one project. 
* Right now, there are multiple rows for each project to account for a project being constructed in 1+ areas.
* Keep the row with the least number of nans for the project id.

In [9]:
project_table2 = _sb1_utils.one_row_one_project(project_table)

In [10]:
len(project_table2)

8261

In [11]:
project_table.projectid.nunique(), project_table.projname.nunique(), project_table.description.nunique()

(7602, 5277, 5062)

In [12]:
project_table2.projectid.nunique(), project_table2.projname.nunique(), project_table2.description.nunique()

(7602, 5277, 5062)

In [13]:
    project_table2.projectid.value_counts().head()

SccpC1Fy17216065Pp01    2
LppcC1Fy17206507Pp19    1
LsrFy17185163Pp005      1
LsrFy17185278Pp001      1
LsrFy17185243Pp074      1
Name: projectid, dtype: int64

In [14]:
project_table2.projectid.nunique(), project_table2.projname.nunique(), project_table2.description.nunique()

(7602, 5277, 5062)

In [15]:
project_table.loc[project_table.projectid == "0200020286"].drop(columns=dont_preview)

,projectid,projname,projcatcode,projcategory,projprogcode,projprogram,multiprogfunded,projstatus,cost,assemblydistrict,senatedistrict,assemblycode,senatecode,countyname,cityname,countycode,citycode,appagencyname,impagencyname,totalcosts,routes,constyear,costfull,projagency,number_of_nans
8682,0200020286,Lassen Modoc 299 Pavement Rehab Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,25200000,01,01,01,01,Lassen Modoc,NaN,Lasmod,NaN,NaN,NaN,NaN,299,2020,25161000.00,Caltrans,7
732,0200020286,Lassen Modoc 299 Pavement Rehab Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,25200000,01,01,01,01,Lassen Modoc,,Lasmod,,NaN,NaN,NaN,299,2020,25161000.00,Caltrans,4
244,0200020286,Lassen Modoc 299 Pavement Rehab Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,25200000,01,01,01,01,Lassen Modoc,,Lasmod,,NaN,NaN,NaN,299,2020,25161000.00,Caltrans,4


In [16]:
project_table2.loc[project_table2.projectid == "0200020286"].drop(columns="description")

,projectid,projname,projcatcode,projcategory,projprogcode,projprogram,multiprogfunded,projstatus,cost,assemblydistrict,senatedistrict,assemblycode,senatecode,countyname,countycode,appagencyname,impagencyname,totalcosts,routes,constyear,costfull,projagency
199,0200020286,Lassen Modoc 299 Pavement Rehab Project,State,State,Shp,State Highway Projects,NaN,PreConstruction,25200000,01,01,01,01,Lassen Modoc,Lasmod,NaN,NaN,NaN,299,2020,25161000.00,Caltrans


In [17]:
project_table2.loc[project_table2.projectid == "LppcC1Fy17206507Pp19"].drop(
    columns="description"
)

,projectid,projname,projcatcode,projcategory,projprogcode,projprogram,multiprogfunded,projstatus,cost,assemblydistrict,senatedistrict,assemblycode,senatecode,countyname,countycode,appagencyname,impagencyname,totalcosts,routes,constyear,costfull,projagency
0,LppcC1Fy17206507Pp19,Redlands Passenger Rail,Local,Local And Regional,Lpp,Local Partnership Program,Local Partnership Program 10831000Solutions For Congested Corridors 65000000,Construction,10831000,40 47,20 23,4047,2023,San Bernardino,Sbd,San Bernardino County Transportation Authority,San Bernardino County Transportation Authority,NaN,NaN,2019,10831000.00,San Bernardino County Transportation Authority


In [18]:
project_table2.loc[project_table2.projectid == "TcepC1Fy17206249Pp10"].drop(
    columns="description"
)

,projectid,projname,projcatcode,projcategory,projprogcode,projprogram,multiprogfunded,projstatus,cost,assemblydistrict,senatedistrict,assemblycode,senatecode,countyname,countycode,appagencyname,impagencyname,totalcosts,routes,constyear,costfull,projagency
24,TcepC1Fy17206249Pp10,Rt 8068012 Interchange Package 2A,Local,Local And Regional,Tce,Trade Corridor Enhancement Program,NaN,Construction,53200000,04 11,03,0411,03,Solano,Sol,Solano Transportation Authority,Sta,NaN,NaN,2021,53200000.00,Solano Transportation Authority


### Separate out SB1 based on our table schema

In [19]:
def merge(df: pd.DataFrame, subset_cols: list):
    df2 = df[subset_cols]
    display(
        pd.merge(df, df2, on=subset_cols, how="inner", indicator=True)[
            ["_merge"]
        ].value_counts()
    )
    return df2

#### Assembly Districts

In [20]:
assembly_subset = [
    "objectid",
    "projectid",
    "projname",
    "projagency",
    "assemblycode",
    "assemblydistrict",
]

In [21]:
sb1_df1.columns

NameError: name 'sb1_df1' is not defined

In [ ]:
assembly_df = merge(sb1_df1, assembly_subset)

In [ ]:
assembly_df.sample(3)

#### Awards

In [ ]:
sb1_df1.columns

In [ ]:
awards_subset = [
    "objectid",
    "projectid",
    "projname",
    "projagency",
    "projprogcode",
    "projprogram",
    "multiprogfunded",
]

In [ ]:
awards_df = merge(sb1_df1, awards_subset)

In [ ]:
awards_df.projprogram.value_counts()

#### County

In [ ]:
county_subset = [
    "objectid",
    "projectid",
    "projname",
    "projagency",
    "countyname",
    "countycode",
]

In [ ]:
county_df = merge(sb1_df1, county_subset)

In [ ]:
county_df.sample(3)

#### City

In [ ]:
city_subset = [
    "objectid",
    "projectid",
    "projname",
    "projagency",
    "cityname",
    "citycode",
]

In [ ]:
city_df = merge(sb1_df1, city_subset)

#### Senate District

In [ ]:
sb1_df1.columns

In [ ]:
senate_subset = [
    "objectid",
    "projectid",
    "projname",
    "projagency",
    "senatedistrict",
    "senatecode",
]

In [ ]:
senate_df = merge(sb1_df1, senate_subset)

#### Geometry

In [ ]:
geo_subset = ["objectid", "projectid", "projname", "projagency", "geometry"]

In [ ]:
gdf = merge(sb1_df1, geo_subset)

In [ ]:
type(gdf)

#### Project Base Table

In [ ]:
project_subset = [
    "objectid",
    "projectid",
    "projname",
    "description",
    "cost",
    "totalcosts",
    "routes",
    "costfull",
    "projagency",
    "appagencyname",
    "impagencyname",
]

In [ ]:
project_df = sb1_df1[project_subset]

In [ ]:
# project_df.sample(100)

#### Phase Funding

In [ ]:
phase_subset = [
    "objectid",
    "projectid",
    "projname",
    "projstatus",
    "projagency",
    "constyear",
]

In [ ]:
phase_df = merge(sb1_df1, phase_subset)

In [ ]:
phase_df.head()

In [ ]:
url_pt1 = "https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_092023/FeatureServer/"

In [ ]:
url_pt2 = "/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&resultOffset=&resultRecordCount=&returnTrueCurves=false&sqlFormat=none&f=geojson"

In [ ]:
int_to_string = list(map(str, [*range(0, 28)]))

In [ ]:
full_gdf = pd.DataFrame()
for i in int_to_string:
    df = to_snakecase(gpd.read_file(f"{url_pt1}{i}{url_pt2}"))
    full_gdf = pd.concat([full_gdf, df], axis=0)

In [ ]:
full_gdf.shape

In [ ]:
full_gdf.shape, sb1_df1.shape

#### Compare

In [ ]:
full_gdf.columns

In [ ]:
full_gdf2 = full_gdf[
    ["projecttitle", "projectdescription", "countycodes", "ct_districts"]
]

In [ ]:
sb1_df1.columns

In [ ]:
sb1_df2 = sb1_df1[["projname", "projagency", "description", "countycode"]]

In [ ]:
pd.merge(
    sb1_df2,
    full_gdf2,
    left_on=["projname", "description", "countycode"],
    right_on=["projecttitle", "projectdescription", "countycodes"],
    how="outer",
    indicator=True,
)[["_merge"]].value_counts()

In [ ]:
pd.merge(
    sb1_df2,
    full_gdf2,
    left_on=["projname", "description"],
    right_on=["projecttitle", "projectdescription"],
    how="outer",
    indicator=True,
)[["_merge"]].value_counts()

In [ ]:
pd.merge(
    sb1_df2,
    full_gdf2,
    left_on=["projname", "countycode"],
    right_on=["projecttitle", "countycodes"],
    how="outer",
    indicator=True,
)[["_merge"]].value_counts()

In [ ]:
url_pt1 = "https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/"
url_pt2 = "/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&resultOffset=&resultRecordCount=&returnTrueCurves=false&sqlFormat=none&f=geojson"

In [ ]:
def sb1_basic_cleaning(
    gdf: gpd.GeoDataFrame, project_name_col: str
) -> gpd.GeoDataFrame:
    """
    Perform basic cleaning before joining
    SB1 & Non SHOPP data together.
    """
    # Some project names contain the year. Remove anything after 20..
    gdf[project_name_col] = gdf[project_name_col].str.split("20").str[0]

    # Get rid of | in object cols
    # https://stackoverflow.com/questions/68152902/extracting-only-object-type-columns-in-a-separate-list-from-a-data-frame-in-pand
    string_cols = gdf.select_dtypes(include=["object"]).columns.to_list()
    try:
        for i in string_cols:
            gdf[i] = (
                gdf[i]
                .str.replace("|", "")
                .str.lower()
                .str.replace("[^\w\s]", "")
                .str.strip()
            )
    except:
        pass

    # Try to extract titles from popups
    try:
        df["popup"] = (
            df["popup"]
            .str.split("<br  />")
            .str[1]
            .str.split("20")
            .str[0]
            .str.lower()
            .str.strip()
            .str.replace("[^\w\s]", "")
        )
    except:
        pass

    return gdf

In [ ]:
def load_sb1_all_projects() -> pd.DataFrame:
    """
    Load in all projects layer of SB1 because it
    contains a value for every row in the
    project title column.
    """
    df = f"{url_pt1}22{url_pt2}"

    df = to_snakecase(gpd.read_file(df))

    # No geometry, just drop it
    df = df.drop(columns=["geometry"])

    # Basic cleaning
    df = sb1_basic_cleaning(
        df, "implementingagency", "projecttitle", "projectid", "projectdescription"
    )

    return df

In [ ]:
def sb1_final() -> gpd.GeoDataFrame:
    """
    Layers 0-21 with geographic information
    don't always have project titles for each
    of the projects. Merge these layers with
    layer 22, which does have title information.
    """
    all_projects_subset = [
        "projecttitle",
        "programcodes",
        "totalcost",
        "implementingagency",
        "fiscalyearcode",
    ]

    sb1_geo, missing_geo = load_sb1_rest_server()
    sb1_all_projects = load_sb1_all_projects()[all_projects_subset]

    # Merge
    merge1 = pd.merge(
        sb1_geo,
        sb1_all_projects,
        how="left",
        left_on=["programcodes", "totalcost", "agencies", "fiscalyearcodes"],
        right_on=["programcodes", "totalcost", "implementingagency", "fiscalyearcode"],
    )

    # Fill in missing project titles in sb1_geo with information from
    # sb1_all_projects
    merge1.projecttitle_x = merge1.projecttitle_x.fillna(merge1.projecttitle_y)

    # Fill missing titles with none
    merge1.projecttitle_x = merge1.projecttitle_x.fillna("None")

    # Drop columns
    merge1 = merge1.drop(columns=["projecttitle_y"])

    # Add a new column detailing where this information comes from
    merge1 = merge1.assign(notes=merge1.programcodes + "/" + "SB1")

    merge1 = merge1.fillna(merge1.dtypes.replace({"float64": 0.0, "object": "None"}))
    return merge1

In [ ]:
sb1_final_gdf = sb1_final()

In [ ]:
sb1_final_gdf.projectstatuscodes.value_counts()

In [ ]:
len(sb1_final_gdf), sb1_final_gdf.geometry.nunique()

In [ ]:
sb1_final_gdf.info()

In [ ]:
sb1_final_gdf.projecttitle_x.value_counts().head(10)

In [ ]:
sb1_final_gdf[sb1_final_gdf.projecttitle_x == "major damage restoration"].groupby(
    ["projecttitle_x", "countynames"]
).agg({"objectid": "count"}).head()

In [ ]:
sb1_final_gdf.projectstatuscodes.value_counts()

In [ ]:
sb1_final_gdf.projectstatuscodes.unique()

In [ ]:
sb1_final_gdf.projectstatuses.value_counts()

In [ ]:
sb1_final_gdf[
    (sb1_final_gdf.projecttitle_x == "major damage restoration")
    & (sb1_final_gdf.countynames == "Alameda")
]

In [ ]:
sb1_final_gdf[
    (sb1_final_gdf.projecttitle_x == "major damage restoration")
    & (sb1_final_gdf.countynames == "Alameda")
].plot()

In [ ]:
# sb1_final_gdf[sb1_final_gdf.projecttitle_x == "major damage restoration"].explore()